### Code for generating the geotiffs that hold the training and validation points used for training the Transformer and SpaRTA models

In [ ]:
import numpy as np
import pandas as pd
import rasterio as rio

from functools import partial

from rasterio.transform import from_bounds
from sklearn.model_selection import train_test_split

from sparta.common import Tile, read_image_bounds, tile_bounds, pool_map
from sparta.common import output_meta, write_gtiff, get_proj_rio

In [2]:
train_paths = {
    "image": "leafon{year}.tif",
    "target": "NLCD_{year}_Land_Cover.tif",
   }
train_years = ['2001', '2004', '2006', '2008', '2011', '2013', '2016', '2019']
train_sequence = ['2001', '2001', '2001', '2004', '2004', '2006', '2006', '2008', '2008', '2008', '2011', '2011', '2013', '2013', '2013', '2016', '2016', '2016', '2019']

In [3]:
ard_tiles = [Tile(3, 10), Tile(4, 1), Tile(13, 13), Tile(20, 8), Tile(24, 13)]

In [4]:
def tile_target_points(bounds: rio.coords.BoundingBox) -> pd.DataFrame:
    """
    Creates a dataframe of x, y geopoints and the corresponding class label given bounds
    """
    train, _ = read_image_bounds(train_paths['target'].replace('{year}', '2016'), bounds)
    xx, yy = np.meshgrid(np.arange(bounds.left, bounds.right, 30), np.arange(bounds.top, bounds.bottom, -30), indexing='ij')
    return pd.DataFrame({'x': np.round(xx[:5000, :5000].ravel()).astype(int), 'y': np.round(yy[:5000, :5000].ravel()).astype(int),
                          'train': train.squeeze().T[:5000, :5000].ravel()})

def get_sampled_points(tile_points):
    """
    Samples points from all ard tiles -> strategy_options applied PER TILE hence (class_max * ntiles)
    """
    data = tile_points[tile_points['train'] != 0]
    data = data.sample(frac=1, random_state=42).reset_index(drop=True)
    samples = data.sample(frac=min(3000000 / len(data), 1), random_state=42)
    other = data[~(data.index.isin(samples.index))]

    nsamples = len(samples)
    min_samples = int(0.02 * nsamples)
    extra_samples = {}
    for key, val in samples['train'].value_counts().to_dict().items():
        if val < min_samples:
            extra_samples[key] = min_samples - val
    extra_data = []
    if len(extra_samples) > 0:
        print("need extra")
        print(extra_samples)
        for key, val in extra_samples.items():
            odata = other[other['train'] == key].sample(frac=1, random_state=42)
            sample_rate = min(1, val / len(odata))
            extra_data.append(odata.sample(frac=sample_rate, random_state=42))
        final_samples = pd.concat([samples, pd.concat(extra_data)]).reset_index(drop=True)

        return final_samples[['x', 'y', 'train']]
    print("no extra")
    return samples[['x', 'y', 'train']]


def convert_coord(sample: tuple, affine: tuple) -> np.array:
    """
    Converts geographic points to array indexes
    """
    row, col = rio.transform.rowcol(affine, sample[0], sample[1])
    return np.array([row, col])

def convert_coords(affine: tuple, coords: list) -> np.ndarray:
    """
    Converts list of coordinates into array indexes
    """
    return np.array(pool_map(partial(convert_coord, affine=affine), coords))

def out_array(bounds: rio.coords.BoundingBox, train_inds: np.ndarray, val_inds: np.ndarray) -> np.ndarray:
    """
    Output array for union of all tile bounds -> writes training points : 1,  validation points: 2
    """
    array = np.zeros((int((bounds.top - bounds.bottom) / 30),
                       int((bounds.right - bounds.left) / 30)))
    array[(train_inds[:, 0], train_inds[:, 1])] = 1
    array[(val_inds[:, 0], val_inds[:, 1])] = 2
    return array

In [5]:
for tile in ard_tiles:
    print(tile)
    bounds = tile_bounds(tile.h, tile.v)
    all_points = tile_target_points(bounds)
    sampled_points = get_sampled_points(all_points)
    X = sampled_points[['x', 'y']]
    y = sampled_points['train']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    train_geos = list(X_train.to_numpy())
    val_geos = list(X_test.to_numpy())
    affine = from_bounds(*bounds, 5000, 5000)
    train_inds = convert_coords(affine, train_geos)
    val_inds = convert_coords(affine, val_geos)
    oarray = out_array(bounds, train_inds, val_inds).astype(np.uint8)
    write_gtiff(oarray, f'./samples/h{tile.h:02}v{tile.v:02}_sample_points.tif', output_meta(affine, get_proj_rio(train_paths['target'].replace('{year}', '2016'))))

Tile(h=3, v=10)
need extra
{46: 5792, 22: 10649, 24: 40526, 11: 42349, 81: 47234, 95: 48489, 41: 50894, 43: 54285, 90: 55248, 12: 57857}
100%|█████████████████████████████████████████████████████████████████████████████████| 667599/667599 [00:39<00:00, 16997.54it/s]
Tile(h=4, v=1)
need extra
{46: 5460, 23: 9886, 90: 16452, 95: 23713, 12: 26049, 24: 42377}
100%|█████████████████████████████████████████████████████████████████████████████████| 624788/624788 [00:36<00:00, 16889.36it/s]
Tile(h=13, v=13)
need extra
{11: 17026, 42: 24328, 22: 37834, 95: 42136, 81: 49565, 23: 49832, 41: 53986, 24: 56149, 31: 56284, 45: 59091, 90: 59579, 43: 59841, 46: 59866}
100%|█████████████████████████████████████████████████████████████████████████████████| 666222/666222 [00:38<00:00, 17287.14it/s]
Tile(h=20, v=8)
need extra
{23: 1395, 11: 9537, 43: 11747, 90: 18972, 24: 40495, 95: 49622, 71: 52732, 31: 57278, 42: 58812, 46: 59188, 45: 59382, 52: 59810}
100%|███████████████████████████████████████████████